# Exploration and download

In this notebook we are going to explore the area of interest and download Sentinel-2 images in a defined time interval, using the EO-TDL library.

In [1]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

## Explore the AoI

First of all, let's explore the area of interest of this workshop. In this case we have chosen the [Boadella reservoir](https://es.wikipedia.org/wiki/Embalse_de_Darnius_Boadella) in Catalonia, Spain, which geometry is in the data folder as `data/boadella.geojson`. 

To do the exploration we are going to use [leafmap](https://leafmap.org/). As we want to make the first zoom to our AoI we need the centroid of the geoJSON file, which will also help us later to extract the bounding box of the images to download.

In [2]:
import leafmap
import geopandas as gpd

in_geojson = 'data/boadella.geojson'
gdf = gpd.read_file(in_geojson)

centroid_coords = gdf['geometry'].centroid
centroid = [centroid_coords.y.values[0], centroid_coords.x.values[0]]   # We are going to use the centroid later

m = leafmap.Map(center=centroid, zoom=13)
m.add_geojson(in_geojson, layer_name="Boadella reservoir")
m

Map(center=[42.347577325903515, 2.815024677909404], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Create bounding box

Images 512x512 10m

In [3]:
from eotdl.tools.tools import bbox_from_centroid

boadella_bbox = bbox_from_centroid(x=centroid[0], y=centroid[1], pixel_size=10, width=512, height=512)

Visualize bounding box

In [4]:
from eotdl.tools.tools import bbox_to_polygon

boadella_polygon = bbox_to_polygon(boadella_bbox)
# comment...
gdf = gpd.GeoDataFrame(geometry=[boadella_polygon])
# comment...
# gdf.to_file('data/boadella_bbox.geojson', driver='GeoJSON')   # Uncomment to save the bbox as a GeoJSON file

m.add_geojson('data/boadella_bbox.geojson', layer_name="Boadella bbox")
m

Map(center=[42.347577325903515, 2.815024677909404], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Explore Sentinel-2 available data

In [5]:
from os import getenv
from eotdl.access import SHClient

sh_client_id = getenv('SH_CLIENT_ID')
sh_client_secret = getenv('SH_CLIENT_SECRET')

client = SHClient(sh_client_id=sh_client_id, 
                  sh_client_secret=sh_client_secret)

> Note: If you already had a Sentinel HUB account before accepting the Terms and Conditions, your SH credentials will NOT appear here (you can retrieve them from you Sentinel HUB dashboard).

Range of dates (see archive `data/dates.csv` ...)

In [15]:
import csv

dates = []
with open("data/dates.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        dates.append(row[0])
dates.sort()

print(dates)

['2019-06-02', '2019-06-07', '2019-06-17', '2019-06-27', '2019-07-02', '2019-07-12', '2019-07-17', '2019-07-22', '2019-08-01', '2019-08-11', '2019-08-16', '2019-08-21', '2019-08-26', '2019-08-31', '2019-09-05', '2019-09-15', '2019-09-30', '2019-10-15', '2019-10-25', '2019-10-30', '2019-11-04', '2019-11-09', '2019-11-24', '2019-11-29', '2019-12-09', '2020-01-13', '2020-01-28', '2020-02-02', '2020-02-17', '2020-02-22', '2020-03-08', '2020-05-07', '2020-05-22', '2020-05-27', '2020-06-21', '2020-07-01', '2020-07-06', '2020-07-11', '2020-07-21', '2020-07-26', '2020-07-31', '2020-08-05', '2020-08-10', '2020-08-15', '2020-08-20', '2020-08-25', '2020-08-30', '2020-09-04', '2020-09-09', '2020-09-14', '2020-10-19', '2020-10-24', '2020-10-29', '2020-11-08', '2020-11-18', '2020-11-23', '2020-12-03', '2020-12-28', '2021-01-12', '2021-01-27', '2021-02-26', '2021-03-18', '2021-03-23', '2021-03-28', '2021-04-02', '2021-04-07', '2021-04-12', '2021-05-12', '2021-05-27', '2021-06-21', '2021-06-26', '2021

We need a dict with format ...
Although we have the dates, we are going to search for the entire time interval ...

In [16]:
from eotdl.tools import get_available_data_by_location

boadella_search_dict = {
    'Boadella': {
        'bounding_box': boadella_bbox,
        'time_interval': (dates[0], dates[-1])
    }
}

boadella_available_data, boadella_not_available_data = get_available_data_by_location(boadella_search_dict, 
                                                         eotdl_client=client,
                                                         sentinel_mission='sentinel-2')

The result is a dict with a list of dates with available images (...)

In [17]:
dates = boadella_available_data['Boadella']['time_interval']
dates[:5]

2023-09-19
2023-09-14
2023-09-09
2023-09-04
2023-08-30


In [ ]:
print(len(dates))

We got a list with dates with available Sentinel-2 images for our AoI. Eventhough we can use it, we already have a list of dates with available data we know is usable for the workshop (no cloud, etc) (...). 

Let's continue in the `01_download` notebook and download the images!